<a href="https://colab.research.google.com/github/hansong0219/Advanced-Deep-learning-Notebooks/blob/master/Cross-Domain/CycleGAN_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReflectionPadding2D

CycleGAN 논문 및 Source 를 통해 ResNet 생성기의 Layer 구조에서 Reflection Padding을 사용하는 것이 ZerosPadding을 사용하는 Conv2D의 padding=same 조건에 비해 권장 되었다. 따라서 ResNet 생성기에는 ReflectionPadding2D의 Class를 지정하여 사용한다.

In [ ]:
class ReflectionPadding2D(layers.Layer):
    # ReflectionPadding Layer 를 실행할 수 있도록 구성된 class
    
    # 입력인수
    """
    padding(tuple): padding을 위한 특정 차원의 크기로 지정된다. 
    """
    # 출력 : padding 이 실행된 텐서를 출력한다.

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")